In [9]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [44]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [45]:
%run -i ../../core.py

In [46]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [47]:
x1_vals_path = os.path.join('x1_vals.json')
x2_vals_path = os.path.join('x2_vals.json')
dummy_x1_vals_path = os.path.join('shared/dummy_x1_vals.json')
dummy_x2_vals_path = os.path.join('shared/dummy_x2_vals.json')

x1_vals= np.array(json.loads(open(x1_vals_path, "r").read())['input_data'][0])
dummy_x1_vals = np.random.uniform(min(x1_vals), max(x1_vals), len(x1_vals))
json.dump({"input_data":[dummy_x1_vals.tolist()]}, open(dummy_x1_vals_path, 'w'))

x2_vals= np.array(json.loads(open(x2_vals_path, "r").read())['input_data'][0])
dummy_x2_vals = np.random.uniform(min(x2_vals), max(x2_vals), len(x2_vals))
json.dump({"input_data":[dummy_x2_vals.tolist()]}, open(dummy_x2_vals_path, 'w'))


y_vals_path = os.path.join('y_vals.json')
dummy_y_vals_path = os.path.join('shared/dummy_y_vals.json')

y_vals= np.array(json.loads(open(y_vals_path, "r").read())['input_data'][0])
dummy_y_vals = np.random.uniform(min(y_vals), max(y_vals), len(y_vals))
json.dump({"input_data":[dummy_y_vals.tolist()]}, open(dummy_y_vals_path, 'w'))


def stacked_x(*args):
    result = np.column_stack((*args, np.ones_like(args[0])))
    return result

x_one = stacked_x(x1_vals, x2_vals)
dummy_x_one = stacked_x(dummy_x1_vals, dummy_x2_vals)

w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(x_one.transpose(), x_one)), x_one.transpose()), y_vals)
dummy_w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(dummy_x_one.transpose(), dummy_x_one)), dummy_x_one.transpose()), dummy_y_vals)

print("reg fit: ", w_vals)
w_tensor = torch.reshape(torch.tensor(w_vals), (1,3,1))
dummy_w_tensor = torch.reshape(torch.tensor(dummy_w_vals), (1,3,1))


reg fit:  [1.98835287 3.14238058 0.06413647]


In [48]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_w_tensor, requires_grad = False)

    def forward(self, *args):
        # infer Y from the last parameter
        Y = args[-1]
        X_one = torch.cat((*args[:-1], torch.ones_like(args[0][:, :, -1:])), dim=2)
        X_T = torch.transpose(X_one, 1, 2)
        # some expression of tolerance to error in the inference
        return (
            torch.sum(torch.abs(X_T @ X_one @ self.w.to('cpu') - X_T @ Y)) < 0.01 * torch.sum(X_T @ Y),
            self.w.to('cpu')
        )
    

verifier_define_calculation(verifier_model, verifier_model_path, [dummy_x1_vals_path, dummy_x2_vals_path, dummy_y_vals_path])

In [54]:
# prover calculates settings, send to verifier

theory_output = w_tensor
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.w = nn.Parameter(data = w_tensor, requires_grad = False)

    def forward(self, *args):
        # infer Y from the last parameter
        Y = args[-1]
        X_one = torch.cat((*args[:-1], torch.ones_like(args[0][:, :, -1:])), dim=2)
        X_T = torch.transpose(X_one, 1, 2)
        # some expression of tolerance to error in the inference
        return (
            torch.sum(torch.abs(X_T @ X_one @ self.w.to('cpu') - X_T @ Y)) < 0.01 * torch.sum(X_T @ Y),
            self.w.to('cpu')
        )

prover_gen_settings([x1_vals_path, x2_vals_path, y_vals_path], comb_data_path, prover_model,prover_model_path, [3], "resources", settings_path)

Theory output:  tensor([[[1.9884],
         [3.1424],
         [0.0641]]], dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  [3]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":3,"param_scale":3,"scale_rebase_multiplier":1,"lookup_range":[0,742824],"logrows":20,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":11808,"total_assignments":795,"total_const_size":0,"model_instance_shapes":[[1],[1,3,1]],"model_output_scales":[0,3],"model_input_scales":[3,3,3],"module_sizes":{"kzg":[],"poseidon":[11808,[3]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":8.0}},{"Div":{"denom":100.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [55]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 107.16260099411011 seconds
Theory output:  tensor([[[1.9884],
         [3.1424],
         [0.0641]]], dtype=torch.float64)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 2.0
witness result 2 : 3.125
witness result 3 : 0.125
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[13372645934914757851, 4179548409246906955, 7308753190278705352, 585334253871252673], [14296875065875978922, 10817153402807448554, 16646795345921275234, 1848850777343635446], [11033437535517255964, 15518297366369924990, 13029218448037200835, 1837434524902037587], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [13392416068245389228, 14913608834158628949, 9631264589208289646, 2240050767903289935], [18409976096712818556, 4988926951396865304, 17770093507857248247, 884830624746662524], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287]]], 'proof': '28dfedf5b1e11c1b4e0b355dd7b82b0fa415f5d829a2f0eb9c2ff9ee435d09b81aefbb906f4ac7c6e162ba4a4e6280267674238ceef8091d8335c61998bf8c1f1e07123044aee2d35d6a244c77bfcaf2dc8bb47ffc15861d2aa03f9d705495a603fa0a03e886958622d348cc6dd6df0320540425f573de70dbbc04256fb9bbbe09ae96419ee08a8d96b2b5c2da0c4b7507cadfe28ecb53bab008ad4e4582094106d66c22bf0a16faec249ca17a600ea

In [56]:
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  3
prf instances:  [[[13372645934914757851, 4179548409246906955, 7308753190278705352, 585334253871252673], [14296875065875978922, 10817153402807448554, 16646795345921275234, 1848850777343635446], [11033437535517255964, 15518297366369924990, 13029218448037200835, 1837434524902037587], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [13392416068245389228, 14913608834158628949, 9631264589208289646, 2240050767903289935], [18409976096712818556, 4988926951396865304, 17770093507857248247, 884830624746662524], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287]]]
proof boolean:  1.0
proof result 1 : 2.0
proof result 2 : 3.125
proof result 3 : 0.125
verified
